Import Library

In [1]:
%matplotlib inline
from oceanum.datamesh import Connector
from datetime import datetime,timedelta
import os,glob
import xarray as xr
# import oceantide
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.rcParams['figure.figsize'] = [10, 5]

Connect to the datamesh using previously saved TOKEN

In [2]:
# datamesh=Connector(token="8cc5a0ee065ee7ff29ae7019bfff4c1edec6adb5") # Cawthron
datamesh=Connector(token="f18191909cc6eec328b1b0851428163b9e0c19af") # Sea Spurge

You are using version 1.0.12 of oceanum_python. A new version is available: 1.0.13. Please update your client to benefit from the latest features and updates.
Using datamesh API version 1


In [3]:
AREA=[105,-52,186,-8] #geographic area
TIME=pd.date_range(datetime(2011,1,1),datetime(2012,1,1),freq='3h')# final timeseries
DELT=timedelta(days=10) #Frequency to download the data could be daily or monthly

#datasets
CURRENT_MODEL={
    'datasource':'glorys_reanalysis_surface_12v1',
    'variables':['depth','uo','vo'],
}
WIND_MODEL={
    'datasource':'era5_wind10m',
    'variables':['u10','v10'],
}                 
TIDE_MODEL={
    # 'datasource':'tpxo9_v5a_cons',
    # 'datasource':'oceanum_global_multiscale_tidegrid',
    'datasource':'oceanum_global_multiscale_tidegrid_timeseries',
    'variables':['u','v'],
} 


# Download the datasets

## The statics data

In [4]:
import copernicusmarine as cm

cm.subset(
  dataset_id="cmems_mod_glo_phy_anfc_0.083deg_static",
  variables=["deptho", "mask","deptho_lev"],
  minimum_longitude=AREA[0],
  maximum_longitude=AREA[2],
  minimum_latitude=AREA[1],
  maximum_latitude=AREA[3],
  output_filename = "/data4/ls/hindcasts/GLORYS/Australia_global_2D_surface_only/glorys_static.nc",
  username='rzyngfogel',
  password='!5Lb51fvP8av',
)

INFO - 2025-10-23T04:36:53Z - Selected dataset version: "202211"
INFO - 2025-10-23T04:36:53Z - Selected dataset part: "bathy"
INFO - 2025-10-23T04:36:56Z - Starting download. Please wait...


  0%|          | 0/109 [00:00<?, ?it/s]

INFO - 2025-10-23T04:37:12Z - Successfully downloaded to /data4/ls/hindcasts/GLORYS/Australia_global_2D_surface_only/glorys_static_(1).nc


ResponseSubset(file_path=PosixPath('/data4/ls/hindcasts/GLORYS/Australia_global_2D_surface_only/glorys_static_(1).nc'), output_directory=PosixPath('.'), filename='glorys_static_(1).nc', file_size=491.15517557251906, data_transfer_size=848.4660763358779, variables=['deptho', 'mask', 'deptho_lev'], coordinates_extent=[GeographicalExtent(minimum=105.0, maximum=186.0, unit='degrees_east', coordinate_id='longitude'), GeographicalExtent(minimum=-52.0, maximum=-8.0, unit='degrees_north', coordinate_id='latitude'), GeographicalExtent(minimum=0.49402499198913574, maximum=5727.9169921875, unit='m', coordinate_id='depth')], status='000', message='The request was successful.', file_status='DOWNLOADED')

## the wind

In [5]:
tstart=TIME[0]
while tstart<=TIME[-1]:
    filename='wind_'+tstart.strftime('%Y%m%dT%H.nc')
    if not os.path.exists(filename):
        print('Downloading %s'% filename)

        query = {
                "datasource": WIND_MODEL['datasource'],
                "timefilter": {
                    "type": "range",
                    "times": [tstart, tstart+DELT]
                },
                "geofilter":{
                    "type": "bbox",
                    "geom": AREA,
                    "resolution": 0.0,

                    },
                "variables": WIND_MODEL['variables']
            }
        dset = datamesh.query(query,)
        dset.to_netcdf(filename)
    
    tstart+=DELT

## The currents

In [4]:
tstart=TIME[0]
while tstart<=TIME[-1]:
    filename='current_'+tstart.strftime('%Y%m%dT%H.nc')
    if not os.path.exists(filename):
        datamesh=Connector(token="8cc5a0ee065ee7ff29ae7019bfff4c1edec6adb5")
        print('Downloading %s'% filename)
        
        query1 = {
                "datasource": CURRENT_MODEL['datasource'],
                "timefilter": {
                    "type": "range",
                    "times": [tstart, tstart+DELT]
                },
                "geofilter":{
                    "type": "bbox",
                    "geom": [AREA[0],AREA[1],180,AREA[3]],
                    "resolution": 0.0,

                    },
                "variables": CURRENT_MODEL['variables']
            }
        dset1 = datamesh.query(query1,)
        query2 = {
                "datasource": CURRENT_MODEL['datasource'],
                "timefilter": {
                    "type": "range",
                    "times": [tstart, tstart+DELT]
                },
                "geofilter":{
                    "type": "bbox",
                    "geom": [-180,AREA[1],-174,AREA[3]],
                    "resolution": 0.0,

                    },
                "variables": CURRENT_MODEL['variables']
            }
        dset2 = datamesh.query(query2,)
        dset2=dset2.assign_coords(longitude=360+dset2['longitude'][:] )
        
        dset=xr.concat([dset1,dset2],dim='longitude')
        dset.to_netcdf(filename)
    else:
        print('%s already exists'% filename)
    tstart+=DELT
    

current_20110101T00.nc already exists
current_20110111T00.nc already exists
current_20110121T00.nc already exists
current_20110131T00.nc already exists
current_20110210T00.nc already exists
current_20110220T00.nc already exists
current_20110302T00.nc already exists
current_20110312T00.nc already exists
current_20110322T00.nc already exists
current_20110401T00.nc already exists
current_20110411T00.nc already exists
current_20110421T00.nc already exists
current_20110501T00.nc already exists
current_20110511T00.nc already exists
current_20110521T00.nc already exists
current_20110531T00.nc already exists
current_20110610T00.nc already exists
current_20110620T00.nc already exists
current_20110630T00.nc already exists
current_20110710T00.nc already exists
current_20110720T00.nc already exists
current_20110730T00.nc already exists
current_20110809T00.nc already exists
current_20110819T00.nc already exists
current_20110829T00.nc already exists
current_20110908T00.nc already exists
current_2011

## The tide

### Download the raw constituent for the area

In [4]:
# filename='tidal_cons.nc'
# print('Downloading %s'% filename)
    
# query = {
#         "datasource": TIDE_MODEL['datasource'],
#         "geofilter":{
#             "type": "bbox",
#             "geom": AREA,
#             "resolution": 0.0,
#             },
#         'variables': ['u', 'v']
#     }
query = {
  'datasource': 'oceanum_global_multiscale_tidegrid',
  'geofilter': {'type': 'bbox', 'geom': [105, -52, 186, -8], 'resolution': 0.0},
  'variables': ['u', 'v']
  }
dset = datamesh.query(query,) # don't save the cons it is too big
dset=dset.drop_dims('time')



/home/ls/miniforge3/envs/oceantracker-seaspurge/lib/python3.11/site-packages/oceanum/datamesh/connection.py:344: UserWarning: Query is too large for direct access, using lazy access with dask
  warnings.warn(


## Tide processing

### Interpolate the cons to the current grid

In [ ]:
current_file=glob.glob('current_*')[0] # just take one wind file to find the grid
nc=xr.open_dataset(current_file)
lat=nc['latitude']
lon=nc['longitude']

: 

In [ ]:
# I think we can use nearest here because tide is at the highest resolution
dset=dset.interp(lon=lon,method='nearest').interp(lat=lat,method='nearest').compute().load()

In [ ]:
dset['u']=dset['u_re']+dset['u_im']*1j
dset['v']=dset['v_re']+dset['v_im']*1j
dset=dset.drop_vars(['u_re','u_im','v_re','v_im']).load()

### Save the timeseries of tide in DELT files for easier processing

In [ ]:
tstart=TIME[0]
while tstart<=TIME[-1]:
    filename='tide_'+tstart.strftime('%Y%m%dT%H.nc')
    tide_time=pd.date_range(tstart,tstart+DELT,freq=TIME.freq.freqstr)
    ts=dset.tide.predict(tide_time,components=[ "u", "v"]).compute() #recreate the timeseries
    ts=ts.rename({'u':'ut','v':'vt'})
    ts.to_netcdf(filename)
    print('Saving %s'% filename)
    tstart+=DELT

/home/remy/software/oceantide/oceantide/core/t_getconsts.py:81: RuntimeWarning: invalid value encountered in cast
  shallow_m1 = const['ishallow'].astype(int) -1
/home/remy/software/oceantide/oceantide/core/t_vuf.py:101: RuntimeWarning: invalid value encountered in cast
  shallow_m1  = const['ishallow'].astype(int) -1


Saving tide_20100101T00.nc


/home/remy/software/oceantide/oceantide/core/t_getconsts.py:81: RuntimeWarning: invalid value encountered in cast
  shallow_m1 = const['ishallow'].astype(int) -1
/home/remy/software/oceantide/oceantide/core/t_vuf.py:101: RuntimeWarning: invalid value encountered in cast
  shallow_m1  = const['ishallow'].astype(int) -1


Saving tide_20100102T00.nc


/home/remy/software/oceantide/oceantide/core/t_getconsts.py:81: RuntimeWarning: invalid value encountered in cast
  shallow_m1 = const['ishallow'].astype(int) -1
/home/remy/software/oceantide/oceantide/core/t_vuf.py:101: RuntimeWarning: invalid value encountered in cast
  shallow_m1  = const['ishallow'].astype(int) -1


...

Saving tide_20101230T00.nc


/home/remy/software/oceantide/oceantide/core/t_getconsts.py:81: RuntimeWarning: invalid value encountered in cast
  shallow_m1 = const['ishallow'].astype(int) -1
/home/remy/software/oceantide/oceantide/core/t_vuf.py:101: RuntimeWarning: invalid value encountered in cast
  shallow_m1  = const['ishallow'].astype(int) -1


Saving tide_20101231T00.nc


/home/remy/software/oceantide/oceantide/core/t_getconsts.py:81: RuntimeWarning: invalid value encountered in cast
  shallow_m1 = const['ishallow'].astype(int) -1
/home/remy/software/oceantide/oceantide/core/t_vuf.py:101: RuntimeWarning: invalid value encountered in cast
  shallow_m1  = const['ishallow'].astype(int) -1


Saving tide_20110101T00.nc


# Iterpolate all the dataset together into DELT files

In [ ]:
current_data=xr.open_mfdataset('current_*.nc',concat_dim="time",combine='nested',data_vars ='minimal').drop_duplicates(dim='time')
wind_data=xr.open_mfdataset('wind_*.nc',concat_dim="time",combine='nested',data_vars ='minimal').drop_duplicates(dim='time')
tide_data=xr.open_mfdataset('tide_*.nc',concat_dim="time",combine='nested',data_vars ='minimal').drop_duplicates(dim='time')


In [ ]:
tstart=TIME[0]
while tstart<=TIME[-1]:
    filename='FullData_'+tstart.strftime('%Y%m%dT%H.nc')
    if not os.path.exists(filename):
        # The tide
        full_data=tide_data.sel(time=slice(tstart,tstart+DELT))
        # the current
        tmp=current_data.interp(time=full_data['time'])
        full_data['u']=tmp['uo'].copy()
        full_data['v']=tmp['vo'].copy()
        full_data['u']+=full_data['ut'][:]
        full_data['v']+=full_data['vt'][:]
        del full_data['ut']
        del full_data['vt']
        # the wind
        tmp=wind_data.interp(time=full_data['time']).interp(longitude=lon).interp(latitude=lat)
        full_data['ugrd10m']=tmp['u10'].copy()
        full_data['vgrd10m']=tmp['v10'].copy()     

        print('Saving %s'% filename)
        full_data.to_netcdf(filename)
    tstart+=DELT

Saving FullData_20100928T00.nc
Saving FullData_20101008T00.nc
Saving FullData_20101018T00.nc
Saving FullData_20101028T00.nc
Saving FullData_20101107T00.nc
Saving FullData_20101117T00.nc
Saving FullData_20101127T00.nc
Saving FullData_20101207T00.nc
Saving FullData_20101217T00.nc
Saving FullData_20101227T00.nc


# Check the data

In [ ]:
full_data=xr.open_mfdataset('FullData_*.nc',concat_dim="time",combine='nested',data_vars ='minimal').drop_duplicates(dim='time')
pt=full_data.isel(longitude=100,latitude=100)

In [ ]:
fig,axs=plt.subplots(3,1)
axs[0].plot(pt['time'],pt['ut'],label='U')
axs[0].plot(pt['time'],pt['vt'],label='V')
axs[0].set_ylabel('Tide vel')
axs[0].xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(axs[2].xaxis.get_major_locator()))

axs[1].plot(pt['time'],pt['uo'],label='U')
axs[1].plot(pt['time'],pt['vo'],label='V')
axs[1].set_ylabel('Res vel')
axs[1].xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(axs[2].xaxis.get_major_locator()))

axs[2].plot(pt['time'],pt['ugrd10m'],label='U')
axs[2].plot(pt['time'],pt['vgrd10m'],label='V')
axs[2].set_ylabel('Wind vel')
axs[2].xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(axs[2].xaxis.get_major_locator()))

plt.legend()
plt.show()

# Clean

In [ ]:
os.system('rm current* wind* tide*')

In [ ]:
all_files=glob.glob('FullData_*')
for file in all_files:
    nc=xr.open_dataset(file)
    nc['uo'][:]+=nc['ut'][:]
    nc['vo'][:]+=nc['ut'][:]
    del nc['ut']
    del nc['vt']
    fileout=file.replace('FullData_','CombineData_')
    nc.to_netcdf(fileout)
    nc.close()
    
    

In [ ]:
full_data=xr.open_mfdataset('CombineData_*.nc',concat_dim="time",combine='nested',data_vars ='minimal').drop_duplicates(dim='time')
pt=full_data.isel(longitude=100,latitude=100)

In [ ]:
fig,axs=plt.subplots(2,1)
axs[0].plot(pt['time'],pt['uo'],label='U')
axs[0].plot(pt['time'],pt['vo'],label='V')
axs[0].set_ylabel('Total vel')
axs[0].xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(axs[0].xaxis.get_major_locator()))

axs[1].plot(pt['time'],pt['ugrd10m'],label='U')
axs[1].plot(pt['time'],pt['vgrd10m'],label='V')
axs[1].set_ylabel('Wind vel')
axs[1].xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(axs[1].xaxis.get_major_locator()))

plt.legend()
plt.show()